In [1]:
import sqlite3
import pandas as pd
import numpy as np
import os

#1. Lấy dữ liệu
def load_data_from_db(db_path):
    print(f"Đang tải dữ liệu từ: {db_path}")
    conn = sqlite3.connect(db_path)

    try:
        # Lấy lịch sử bán hàng
        query_sales = "SELECT product_id, all_time_quantity_sold, crawl_timestamp FROM sales_history"
        df_sales = pd.read_sql_query(query_sales, conn)

        # Lấy thông tin sản phẩm
        query_info = "SELECT id as product_id, name FROM products"
        df_products = pd.read_sql_query(query_info, conn)

        print(f"  ->Đã load {len(df_sales)} dòng sales và {len(df_products)} sản phẩm.")
        return df_sales, df_products
    finally:
        conn.close()

#2. Xử lý & Tính toán
def process_sales_snapshots(df_sales):
    print("Đang xử lý thời gian và lọc dữ liệu...")

    # Chuyển đổi thời gian
    df_sales['crawl_timestamp'] = pd.to_datetime(df_sales['crawl_timestamp'])

    min_date = df_sales['crawl_timestamp'].min()
    max_date = df_sales['crawl_timestamp'].max()
    print(f"Khung thời gian: {min_date} đến {max_date}")

    # Lọc số liệu CUỐI KỲ (Mới nhất)
    df_new = df_sales[df_sales['crawl_timestamp'] == max_date].groupby('product_id')['all_time_quantity_sold'].max().reset_index()
    df_new.rename(columns={'all_time_quantity_sold': 'qty_new'}, inplace=True)

    # Lọc số liệu ĐẦU KỲ (Cũ nhất - Lấy theo ngày)
    first_day = min_date.date()
    df_old = df_sales[df_sales['crawl_timestamp'].dt.date == first_day].groupby('product_id')['all_time_quantity_sold'].max().reset_index()
    df_old.rename(columns={'all_time_quantity_sold': 'qty_old'}, inplace=True)

    return df_new, df_old

def compute_target_variable(df_info, df_new, df_old):
    print("Đang tính toán biến mục tiêu (Target Variable)...")

    df_final = df_info.drop_duplicates(subset=['product_id'], keep='last').copy()

    #1. Ghép Sales
    df_final = pd.merge(df_final, df_new, on='product_id', how='left')
    df_final = pd.merge(df_final, df_old, on='product_id', how='left')

    #2. Xử lý Missing Values
    #Case 1: Sản phẩm mới -> Old = 0
    df_final['qty_old'] = df_final['qty_old'].fillna(0)
    #Case 2: Sản phẩm ẩn/xóa -> New = Old
    df_final['qty_new'] = df_final['qty_new'].fillna(df_final['qty_old'])

    #3. Tính Target
    df_final['quantity_sold_in_period'] = df_final['qty_new'] - df_final['qty_old']

    return df_final

#3. Load
def save_results(df, output_path):
    print(f"Đang lưu file...")

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    cols_to_save = ['product_id', 'name', 'quantity_sold_in_period']
    df[cols_to_save].to_csv(output_path, index=False)

    print("-" * 30)
    print("Top 10 sản phẩm bán chạy nhất:")
    cols_view = ['product_id', 'name', 'qty_old', 'qty_new', 'quantity_sold_in_period']
    display(df[cols_view].sort_values(by='quantity_sold_in_period', ascending=False).head(10))
    print(f"Hoàn tất! File đã lưu tại: {output_path}")

def main():
    DB_PATH = '../data/database/tiki_products_multi.db'
    OUTPUT_PATH = '../data/processed/target_variable.csv'

    df_sales, df_products = load_data_from_db(DB_PATH)

    df_new, df_old = process_sales_snapshots(df_sales)
    df_final = compute_target_variable(df_products, df_new, df_old)

    save_results(df_final, OUTPUT_PATH)

if __name__ == "__main__":
    main()

Đang tải dữ liệu từ: ../data/database/tiki_products_multi.db
  ->Đã load 220764 dòng sales và 4164 sản phẩm.
Đang xử lý thời gian và lọc dữ liệu...
Khung thời gian: 2025-11-02 10:45:20.297907 đến 2025-11-23 22:21:23.833893
Đang tính toán biến mục tiêu (Target Variable)...
Đang lưu file...
------------------------------
Top 10 sản phẩm bán chạy nhất:


,product_id,name,qty_old,qty_new,quantity_sold_in_period
79,767101,Bao cao su Durex Kingtex 12 bao,11851.0,70895.0,59044.0
8,385582,Bao cao su Durex Fetherlite Hộp 12 Bao,0.0,29837.0,29837.0
53,543129,Nước Giặt Quần Áo Cho Bé D-nee - Chai 3000ml (...,206.0,26198.0,25992.0
953,73787185,Sách Những Tù Nhân Của Địa Lý,0.0,23792.0,23792.0
34,505636,Dầu Gội Dưỡng Tóc Siêu Mượt Enchanteur Charmin...,0.0,18026.0,18026.0
2839,274079594,Kem Đánh Răng Colgate Vitamin C Thơm Mát 170G,0.0,17244.0,17244.0
430,24028050,Sữa bột Abbott Pediasure 1.6kg cho trẻ từ 1-10...,0.0,15122.0,15122.0
439,25108217,Sữa rửa mặt ngăn ngừa mụn Acnes Creamy Wash 100g,173170.0,186252.0,13082.0
673,53583472,Bộ Hộp Cơm Giữ Nhiệt Lock&Lock Easy Carry 2L L...,0.0,11934.0,11934.0
41,525590,Sữa Tắm Nước Hoa Enchanteur Sensation 650g,0.0,10670.0,10670.0


Hoàn tất! File đã lưu tại: ../data/processed/target_variable.csv
